In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/competitive-data-science-predict-future-sales/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

path='/kaggle/input/competitive-data-science-predict-future-sales/'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
import warnings
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
df_items = pd.read_csv(path+'items.csv')
df_sample= pd.read_csv(path+'sample_submission.csv')
df_categ = pd.read_csv(path+'item_categories.csv')
df_train = pd.read_csv(path+'sales_train.csv')
df_shops = pd.read_csv(path+'shops.csv')
df_test  = pd.read_csv(path+'test.csv')

In [ ]:
print(df_items.columns)
print(df_sample.columns)
print(df_categ.columns)
print(df_train.columns)
print(df_shops.columns)
print(df_test.columns)

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_train.hist(figsize=(20,20),bins=30)
plt.show()

In [ ]:
def count_nan_values(df, percent_base = 100):
    columns_drop, col2, count_nan2, percent2, type2 = [], [], [], [], []

    for col in df:
        count_nan = df[col].isna().sum()
        
        if count_nan != 0:
            col2.append(col)
            count_nan2.append(count_nan)
            percent2.append(str((count_nan * 100 / df.shape[0]).round(2)) + '%')
            type2.append(df[col].dtype)
            
            if (count_nan * 100 / df.shape[0]).round(2) >= percent_base:
                columns_drop.append(col)

    result = pd.DataFrame({'col': col2,
                            'count_nan': count_nan2,
                            'percent': percent2,
                            'type': type2}).sort_values(['count_nan'], ascending = [False])

    return result, columns_drop

In [ ]:
def one_class(df):
    df.dropna()
    one_class = []
    for var in count_nan_values(df, percent_base = 0)[1]:
        if df[var].nunique() == 1:
            print(var, df[var].dtype)
            one_class.append(var)

In [ ]:
one_class(df_train)

In [ ]:
df_train['date']

In [ ]:
df_train['date']=pd.to_datetime(df_train['date'], format='%d.%m.%Y')

In [ ]:
df_train['date']

In [ ]:
df=df_train[['date','item_price','item_id']].set_index('date')

In [ ]:
df = df[df.item_price != max(df['item_price'])] #delete outlier

In [ ]:
df.groupby('item_id').count().sort_values(by='item_price',ascending=False)

In [ ]:
df_1 = df[df['item_id']==20949].drop('item_id',axis=1)
df_1.head()

In [ ]:
len(df_1)

In [ ]:
plt.style.use('fivethirtyeight')
# Color pallete for plotting
color_pal = [ "#D39200", "#93AA00",
             "#00BA38", "#00C19F", "#00B9E3",
             "#619CFF", "#DB72FB"]
df_1.plot(style='*', figsize=(15,12), color=color_pal[1], title='Sales Price')
plt.show()